# Stark Map

In this notebook

In [1]:
import importlib
import starktools as st

import numpy as np

In [2]:
defects = {
        0 : [0.29665648771, 0.038296666, 0.0075131, -0.0045476],
        1 : [0.06836028379, -0.018629228, -0.01233275, -0.0079527],
        2 : [0.002891328825, -0.006357704, 0.0003367, 0.0008395],
        3 : [0.00044737927, -0.001739217, 0.00010478, 3.31e-05],
        4 : [0.00012714167, -0.000796484, -9.85e-06, -1.9e-05],
        5 : [4.8729846e-05, -0.0004332281, -8.1e-06, 0],
        6 : [2.3047609e-05, -0.0002610672, -4.04e-06, 0]
}

In [3]:
importlib.reload(st)
nmin = 50
nmax = 53
m0 = st.MatrixH0(nmin, nmax, defects)
mH = st.MatrixHs(nmin, nmax, defects)

In [5]:
m0.nmax

53

In [8]:
m0.shape

<bound method Matrix.shape of {50: {0: {50: {0: -0.0002023946677140535, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0}, 51: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0}, 52: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 

In [5]:
m = m0.make_array()
m.shape

(206, 206)

In [ ]:
#CODATA 2014, DOI: 10.1103/RevModPhys.88.035009
c = 299792458.0 ## speed of Heght in vacuum
h = 6.626070040e-34
hbar = 1.054571800e-34

Ry = 10973731.568508
e = 1.6021766208e-19
m_e = 9.10938356e-31
alpha = 7.2973525664e-3
amu = 1.660539040e-27 # Kg   (NIST) unified atomic mass unit 


a_0 = hbar/ (m_e * c * alpha)
mu_B = e * hbar / (2.0 * m_e)


# Helium
mHe = 4.0026*amu
mHeIon = mHe - m_e
reducedMass = (mHeIon * m_e)/(mHeIon+m_e)


a_He  = (a_0*m_e)/reducedMass

R_He = Ry * reducedMass/m_e
E_He = 2*h*c*R_He

In [ ]:
# Lithium
F_He = E_He/(a_He*e)

In [ ]:
m0.shape

<bound method Matrix.shape of {50: {0: {50: {0: -0.0002023946677140535, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0}, 51: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0}, 52: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 

In [ ]:
def map_maker(m0_array, mH_array, field):
    fsteps = field.shape[0]
    neig = m0_array.shape[0]
    
    vals = np.zeros((fsteps, neig))
    for i in range(fsteps):
        fau = field[i] / F_He
        val = (np.linalg.eigvalsh(m0_array + mH_array*fau) * E_He)/(h*c*100)
        for j in range(neig):
            vals[i,j] = val[j]
    return vals

In [ ]:
field = np.linspace(0, 30, 200)# V /m
v = map_maker(m0, mH, field)

TypeError: 'method' object is not subscriptable

In [ ]:
M0

[55.6 45.7 80.3] kilogram
[1.64 1.85 1.77] meter
[3.28 3.7  3.54] meter


TypeError: unsupported operand type(s) for ** or pow(): 'Physical' and 'int'